In [ ]:
import warnings

warnings.filterwarnings("ignore")

In [ ]:
from langchain import hub

prompt_agent = hub.pull('hwchase17/openai-tools-agent')

prompt_agent

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

llm.invoke("HI")

In [ ]:
import subprocess
from langchain_core.tools import tool


@tool
def list_files():
    """List files in current directory"""
    return subprocess.run(["ls"], stdout=subprocess.PIPE).stdout.decode("utf-8")


list_files.invoke({"input": ""})

In [ ]:
llm_with_tool = llm.bind_tools([list_files])

llm_with_tool

In [ ]:
llm_with_tool.invoke("List files in current directory")

In [ ]:
dictionary_with_tool_names = {
    "list_files": list_files,
}

output = llm_with_tool.invoke("List files in current directory")

if output.tool_calls:
    for tool_call in output.tool_calls:
        tool_name = tool_call["name"]
        tool_args = tool_call['args']
        tool_func = dictionary_with_tool_names[tool_name]
        print(tool_func.invoke(tool_args))

Agent = LLM + tools

In [ ]:
from langchain.agents import create_tool_calling_agent, AgentExecutor

In [ ]:
tools = [list_files]

agent = create_tool_calling_agent(llm, tools, prompt_agent)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
agent_executor.invoke({'input': "List files in current directory"})

In [ ]:
agent_executor.invoke({'input': "What kind of files are in the current directory?"})

In [ ]:
import os

os.environ["TAVILY_API_KEY"] = 'YOUR TAVILY API KEY'

In [ ]:
from langchain_community.tools import TavilySearchResults

In [ ]:
search_tool = TavilySearchResults(
    max_results = 5,
    search_depth="advanced",
    include_answer=True,
)

search_tool.invoke('What is the capital of France?')

In [ ]:
@tool
def create_file(file_path: str, content: str):
    """Create a file with the given content"""
    with open(file_path, 'w') as f:
        f.write(content)
    return f"File created at {file_path} with content: {content}"

tools = [create_file, search_tool]

In [ ]:
simple_research_agent = create_tool_calling_agent(llm, tools, prompt_agent)

simple_research_agent_executor = AgentExecutor(agent=simple_research_agent, tools=tools, verbose=True)

simple_research_agent_executor
